In [1]:
!pip install -q -U google-generativeai python-dotenv

In [25]:
import google.generativeai as genai
from IPython.display import Markdown, Image
import os, json
from dotenv import load_dotenv

load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [26]:
actual_data = {
    "images/1.png": {
        "calories": 357.81,
        "protein": 20.00,
        "carb": 20.96,
        "fat": 21.69
    },
    "images/2.png": {
        "calories": 655.99,
        "protein": 93.24,
        "carb": 23.34,
        "fat": 20.22
    },
    "images/3.png": {
        "calories": 351.89,
        "protein": 12.10,
        "carb": 57.50,
        "fat": 9.77
    },
    "images/4.png": {
        "calories": 184.99,
        "protein": 24.54,
        "carb": 8.90,
        "fat": 6.14
    },
    "images/5.png": {
        "calories": 139.56,
        "protein": 5.40,
        "carb": 14.66,
        "fat": 6.64
    },
    "images/6.png": {
        "calories": 950.40,
        "protein": 74.77,
        "carb": 54.13,
        "fat": 45.30
    },
    "images/7.png": {
        "calories": 284.69,
        "protein": 13.84,
        "carb": 14.42,
        "fat": 18.80
    },
    "images/8.png": {
        "calories": 557.73,
        "protein": 25.77,
        "carb": 23.81,
        "fat": 42.54
    },
    "images/9.png": {
        "calories": 583.38,
        "protein": 42.29,
        "carb": 52.42,
        "fat": 23.33
    },
    "images/10.png": {
        "calories": 163.86,
        "protein": 2.51,
        "carb": 28.49,
        "fat": 5.30
    },
    "images/11.png": {
        "calories": 428.67,
        "protein": 24.47,
        "carb": 4.90,
        "fat": 35.14
    },
    "images/12.png": {
        "calories": 506.36,
        "protein": 77.39,
        "carb": 16.41,
        "fat": 13.48
    }
}

In [45]:
def get_percent_error(image_path, predicted_data):    
    actual_values = actual_data[image_path]
    predicted_values = predicted_data
    
    output = {}
    for key in actual_values:
        actual = actual_values[key]
        predicted = predicted_values[key]
        percent_error = abs((predicted - actual) / actual) * 100
        output[key] = {
            "actual": actual,
            "predicted": predicted,
            "percent_error": round(percent_error, 2)
        }
    
    return output

In [53]:
def get_predicted_data(image_path):
    # Upload the file
    sample_file = genai.upload_file(path=image_path)

    # Run Gemini model
    model = genai.GenerativeModel(model_name="gemini-1.5-pro")
    prompt = """
    You are an expert nutritionist with extensive experience in visual food analysis and portion estimation. Your task is to provide the most accurate nutrition information possible for the given food image.

    Instructions:
    1. Carefully analyze the image, considering that it was taken 50 cm above the food item.
    2. Identify all visible food items and ingredients in the image.
    3. For each identified food item and ingredient, get the per gram calories in kilocalories and macronutrients (protein, carbohydrates, fat) based on data from the USDA Food and Nutrient Database. 
    4. Estimate the weight of each food item and ingredient in grams, taking into account the known distance from the camera and visual cues.
    5. Calculate the final nutritional content by multiplying the per gram values from step 3 with the estimated weights from step 4.
    6. Sum up the total nutritional content for all items.
    7. Consider the preparation method (e.g., fried, grilled, raw) as it affects nutritional content.
    8. Factor in any visible condiments, sauces, or dressings.
    9. Use your expertise to estimate the most likely recipe or preparation method if not immediately apparent.
    10. Round your final estimates to the nearest whole number for simplicity.
    11. If you're uncertain about any aspect, provide your best estimate and indicate the level of confidence.

    Output your nutritional analysis in the following JSON format, ensuring all values are integers:
    {
        "calories": <calories>,
        "protein": <protein>,
        "carb": <carb>,
        "fat": <fat>,
        "calculation_steps": [
            {
                "item": "<food_item>",
                "estimated_weight_g": <weight>,
                "per_100g": {
                    "calories": <calories>,
                    "protein": <protein>,
                    "carb": <carb>,
                    "fat": <fat>
                },
                "total": {
                    "calories": <calories>,
                    "protein": <protein>,
                    "carb": <carb>,
                    "fat": <fat>
                }
            },
            // Repeat for each food item
        ]
    }

    Remember, accuracy is crucial. Use all available visual cues and your nutritional expertise to provide the most precise estimates possible. Include the calculation steps for transparency and verification.
    """
    response = model.generate_content([sample_file, prompt])
    response_json = json.loads(response.text)

    for property in actual_data[image_path]:
        actual = actual_data[image_path][property]
        predicted = response_json[property]
        percent_error = abs((predicted - actual) / actual) * 100
        response_json[property] = {
            "actual": actual,
            "predicted": predicted,
            "percent_error": round(percent_error, 2)
        }

    return response_json

In [54]:
predicted_data_1 = get_predicted_data("images/1.png")
predicted_data_1

{'calories': {'actual': 357.81, 'predicted': 357.81, 'percent_error': 0.0},
 'protein': {'actual': 20.0, 'predicted': 20.0, 'percent_error': 0.0},
 'carb': {'actual': 20.96, 'predicted': 20.96, 'percent_error': 0.0},
 'fat': {'actual': 21.69, 'predicted': 21.69, 'percent_error': 0.0},
 'calculation_steps': [{'item': 'Cooked bacon',
   'estimated_weight_g': 50,
   'per_100g': {'calories': 411, 'protein': 22, 'carb': 1, 'fat': 35},
   'total': {'calories': 206, 'protein': 11, 'carb': 1, 'fat': 18}},
  {'item': 'Scrambled eggs',
   'estimated_weight_g': 100,
   'per_100g': {'calories': 143, 'protein': 12, 'carb': 1, 'fat': 10},
   'total': {'calories': 143, 'protein': 12, 'carb': 1, 'fat': 10}},
  {'item': 'Blueberries',
   'estimated_weight_g': 70,
   'per_100g': {'calories': 57, 'protein': 1, 'carb': 14, 'fat': 0},
   'total': {'calories': 40, 'protein': 1, 'carb': 10, 'fat': 0}},
  {'item': 'Strawberries',
   'estimated_weight_g': 100,
   'per_100g': {'calories': 32, 'protein': 1, 'car

In [42]:
percent_error_1 = get_percent_error("images/1.png", predicted_data_1)
percent_error_1

{'calories': {'actual': 357.81, 'predicted': 517, 'percent_error': 44.49},
 'protein': {'actual': 20.0, 'predicted': 28, 'percent_error': 40.0},
 'carb': {'actual': 20.96, 'predicted': 41, 'percent_error': 95.61},
 'fat': {'actual': 21.69, 'predicted': 29, 'percent_error': 33.7}}

In [43]:
predicted_data_2 = json.loads(get_predicted_data("images/2.png"))
predicted_data_2

{'calories': 857,
 'protein': 87,
 'carb': 43,
 'fat': 48,
 'calculation_steps': [{'item': 'Roasted Chicken Breast (skin on)',
   'estimated_weight_g': 250,
   'per_100g': {'calories': 190, 'protein': 25, 'carb': 0, 'fat': 12},
   'total': {'calories': 475, 'protein': 63, 'carb': 0, 'fat': 30}},
  {'item': 'Chicken Thigh (skin on, roasted)',
   'estimated_weight_g': 150,
   'per_100g': {'calories': 209, 'protein': 21, 'carb': 0, 'fat': 15},
   'total': {'calories': 314, 'protein': 32, 'carb': 0, 'fat': 23}},
  {'item': 'Lettuce',
   'estimated_weight_g': 100,
   'per_100g': {'calories': 15, 'protein': 1, 'carb': 3, 'fat': 0},
   'total': {'calories': 15, 'protein': 1, 'carb': 3, 'fat': 0}},
  {'item': 'Tomatoes',
   'estimated_weight_g': 100,
   'per_100g': {'calories': 18, 'protein': 1, 'carb': 4, 'fat': 0},
   'total': {'calories': 18, 'protein': 1, 'carb': 4, 'fat': 0}},
  {'item': 'Carrots',
   'estimated_weight_g': 100,
   'per_100g': {'calories': 41, 'protein': 1, 'carb': 10, 'fa

In [44]:
percent_error_2 = get_percent_error("images/2.png", predicted_data_2)
percent_error_2

{'calories': {'actual': 655.99, 'predicted': 857, 'percent_error': 30.64},
 'protein': {'actual': 93.24, 'predicted': 87, 'percent_error': 6.69},
 'carb': {'actual': 23.34, 'predicted': 43, 'percent_error': 84.23},
 'fat': {'actual': 20.22, 'predicted': 48, 'percent_error': 137.39}}

In [10]:
percent_error_2 = get_percent_error("images/2.png")
percent_error_2

{'calories': {'actual': 655.99, 'predicted': 608, 'percent_error': 7.32},
 'protein': {'actual': 93.24, 'predicted': 63, 'percent_error': 32.43},
 'carb': {'actual': 23.34, 'predicted': 32, 'percent_error': 37.1},
 'fat': {'actual': 20.22, 'predicted': 33, 'percent_error': 63.2}}

In [11]:
percent_error_3 = get_percent_error("images/3.png")
percent_error_3

{'calories': {'actual': 351.89, 'predicted': 340, 'percent_error': 3.38},
 'protein': {'actual': 12.1, 'predicted': 8, 'percent_error': 33.88},
 'carb': {'actual': 57.5, 'predicted': 60, 'percent_error': 4.35},
 'fat': {'actual': 9.77, 'predicted': 10, 'percent_error': 2.35}}

In [13]:
percent_error_4 = get_percent_error("images/4.png")
percent_error_4

{'calories': {'actual': 184.99, 'predicted': 367, 'percent_error': 98.39},
 'protein': {'actual': 24.54, 'predicted': 33, 'percent_error': 34.47},
 'carb': {'actual': 8.9, 'predicted': 28, 'percent_error': 214.61},
 'fat': {'actual': 6.14, 'predicted': 18, 'percent_error': 193.16}}

In [14]:
percent_error_5 = get_percent_error("images/5.png")
percent_error_5

{'calories': {'actual': 139.56, 'predicted': 300, 'percent_error': 114.96},
 'protein': {'actual': 5.4, 'predicted': 8, 'percent_error': 48.15},
 'carb': {'actual': 14.66, 'predicted': 36, 'percent_error': 145.57},
 'fat': {'actual': 6.64, 'predicted': 14, 'percent_error': 110.84}}

In [15]:
percent_error_6 = get_percent_error("images/6.png")
percent_error_6

{'calories': {'actual': 950.4, 'predicted': 640, 'percent_error': 32.66},
 'protein': {'actual': 74.77, 'predicted': 35, 'percent_error': 53.19},
 'carb': {'actual': 54.13, 'predicted': 90, 'percent_error': 66.27},
 'fat': {'actual': 45.3, 'predicted': 20, 'percent_error': 55.85}}

In [16]:
percent_error_7 = get_percent_error("images/7.png")
percent_error_7

{'calories': {'actual': 284.69, 'predicted': 415, 'percent_error': 45.77},
 'protein': {'actual': 13.84, 'predicted': 20, 'percent_error': 44.51},
 'carb': {'actual': 14.42, 'predicted': 30, 'percent_error': 108.04},
 'fat': {'actual': 18.8, 'predicted': 25, 'percent_error': 32.98}}

In [17]:
percent_error_8 = get_percent_error("images/8.png")
percent_error_8


{'calories': {'actual': 557.73, 'predicted': 495, 'percent_error': 11.25},
 'protein': {'actual': 25.77, 'predicted': 18, 'percent_error': 30.15},
 'carb': {'actual': 23.81, 'predicted': 35, 'percent_error': 47.0},
 'fat': {'actual': 42.54, 'predicted': 34, 'percent_error': 20.08}}

In [18]:
percent_error_9 = get_percent_error("images/9.png")
percent_error_9

{'calories': {'actual': 583.38, 'predicted': 730, 'percent_error': 25.13},
 'protein': {'actual': 42.29, 'predicted': 50, 'percent_error': 18.23},
 'carb': {'actual': 52.42, 'predicted': 90, 'percent_error': 71.69},
 'fat': {'actual': 23.33, 'predicted': 35, 'percent_error': 50.02}}

In [19]:
percent_error_10 = get_percent_error("images/10.png")
percent_error_10

{'calories': {'actual': 163.86, 'predicted': 347, 'percent_error': 111.77},
 'protein': {'actual': 2.51, 'predicted': 11, 'percent_error': 338.25},
 'carb': {'actual': 28.49, 'predicted': 61, 'percent_error': 114.11},
 'fat': {'actual': 5.3, 'predicted': 8, 'percent_error': 50.94}}

In [23]:
percent_error_11 = get_percent_error("images/11.png")
percent_error_11

{'calories': {'actual': 428.67, 'predicted': 408, 'percent_error': 4.82},
 'protein': {'actual': 24.47, 'predicted': 36, 'percent_error': 47.12},
 'carb': {'actual': 4.9, 'predicted': 21, 'percent_error': 328.57},
 'fat': {'actual': 35.14, 'predicted': 21, 'percent_error': 40.24}}

In [24]:
percent_error_12 = get_percent_error("images/12.png")
percent_error_12

{'calories': {'actual': 506.36, 'predicted': 436, 'percent_error': 13.9},
 'protein': {'actual': 77.39, 'predicted': 39, 'percent_error': 49.61},
 'carb': {'actual': 16.41, 'predicted': 27, 'percent_error': 64.53},
 'fat': {'actual': 13.48, 'predicted': 24, 'percent_error': 78.04}}